In [27]:
import numpy as np
from statsmodels.sandbox.stats.runs import mcnemar
import cPickle
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import cPickle

In [28]:
base_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/final_experiments/heldout/sanity_check_heldout/"

In [32]:
mcnemar_vectors_dir = base_dir + "mcnemar_vectors/"

# m

In [44]:
mcnemar_supervectors_m_filename = mcnemar_vectors_dir + "supervectors/m"
mcnemar_dct_m_filename = mcnemar_vectors_dir + "dct/m"

In [45]:
with open(mcnemar_supervectors_m_filename, "rb") as f:
    mcnemar_supervectors_m = cPickle.load(f)

In [48]:
with open(mcnemar_dct_m_filename, "rb") as f:
    mcnemar_dct_m = cPickle.load(f)

In [49]:
print sum(mcnemar_supervectors_m['m'])
print sum(mcnemar_dct_m['m'])

805
804


In [50]:
mcnemar(mcnemar_supervectors_m['m'], mcnemar_dct_m['m'])

(49, 1.0)

In [4]:
phonemes = ["i", "m"]

In [124]:
def check_lines(lines_supervectors, lines_dct):
    logids_supervectors = [l[0] for l in lines_supervectors]
    logids_dct = [l[0] for l in lines_dct]
    if not logids_supervectors == logids_dct:
        raise Exception(phoneme + " logids doesn't match in supervectors vs dct")
    labels_supervectors = [t[3] for t in lines_supervectors]
    labels_dct = [t[3] for t in lines_supervectors]
    if not labels_supervectors == labels_dct:
        raise Exception(phoneme + " labels doesn't match in supervectors vs dct")

In [125]:
def all_phonemes_empty_dict(phonemes):
    return dict(zip(phonemes, [None for p in phonemes]))

In [126]:
def label_for(label):
    if label == "True":
        return 1
    elif label == "False":
        return -1

In [127]:
def load_positives_negatives_dict():
    phonemes = ["b", "m", "B", "s", "D", "G", "w", "i", "y", "p", "r", "z", "t", "k", "rr", "a", "o", "l", "e", "n", "u", "c", "x", "d", "g", "N"]
    positive_values = [140, 801, 126, 1963, 268, 61, 179, 1224, 596, 441, 910, 49, 733, 434, 122, 2509, 2030, 851, 2658, 1792, 471, 104, 161, 191, 237, 246]
    positive_values = [str(p) for p in positive_values]
    negative_values = [95, 188, 302, 107, 486, 145, 128, 301, 156, 254, 641, 247, 360, 388, 453, 548, 548, 356, 899, 125, 110, 24, 37, 18, 29, 116]
    negative_values = [str(n) for n in negative_values]
    positives_dict = dict(zip(phonemes, positive_values))
    negatives_dict = dict(zip(phonemes, negative_values))
    return (positives_dict, negatives_dict)

In [128]:
def score_combination_dict():
    return {
        'Y': 0.8,
        'c': 0.9,
        'f': 0.4,
        'x': 0.6,
        'd': 0.2,
        'G': 0.1,
        'b': 0.1,
        'g': 0.1,
        'z': 0.3,
        'w': 0.0,
        'N': 1.0,
        'B': 0.3,
        'rr': 0.9,
        'u': 1.0,
        'p': 0.0,
        'D': 0.1,
        'y': 1.0,
        'k': 0.2,
        'm': 0.6,
        't': 0.6,
        'l': 0.8,
        'i': 0.3,
        'r': 0.6,
        'n': 0.7,
        's': 1.0,
        'o': 0.9,
        'a': 0.7,
        'e': 0.6
    }

In [129]:
def compute_decision_vectors(per_phoneme_values):
    labels = [t[1] for t in per_phoneme_values]
    llrs = [t[0] for t in per_phoneme_values]
    fpr, tpr, thresholds = roc_curve(labels, llrs)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    fpr_eer_diff = abs(fpr - eer)
    min_index = np.argmin(fpr_eer_diff, axis=0)
    threshold = thresholds[min_index]
    decisions = llrs > threshold
    labels = [l == True for l in labels]
    return (decisions == labels)

# Score combination

In [130]:
score_combination_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/final_experiments/heldout/score_combination/"
supervectors_single_phonemes_dir = score_combination_dir + "supervectors_single_phonemes/"
dct_single_phonemes_dir = score_combination_dir + "dct_single_phonemes/"

In [148]:
mcnemar_dict = all_phonemes_empty_dict(phonemes)
decision_vectors_supervectors_dict = all_phonemes_empty_dict(phonemes)
for phoneme in phonemes:
    combination_dict = score_combination_dict()[phoneme]
    original_phoneme = phoneme
    if phoneme in ['g', 'y', 'd', 'b', 'n']:
        phoneme = phoneme + "_lowercase"
    supervectors_single_phoneme_filename = supervectors_single_phonemes_dir + phoneme
    with open(supervectors_single_phoneme_filename, "r") as f:
        lines_supervectors = f.readlines()
        lines_supervectors = [l.strip().split(" ") for l in lines_supervectors]
    dct_single_phoneme_filename = dct_single_phonemes_dir + phoneme
    with open(dct_single_phoneme_filename, "r") as f:
        lines_dct = f.readlines()
        lines_dct = [l.strip().split(" ") for l in lines_dct]
    check_lines(lines_supervectors, lines_dct)
    labels = [label_for(t[3]) for t in lines_supervectors]
    per_phoneme_values_combination = []
    per_phoneme_values_supervectors = []
    for i in range(len(lines_supervectors)):
        supervectors_value = float(lines_supervectors[i][4])
        dct_value = float(lines_dct[i][4])
        per_phoneme_values_combination.append((supervectors_value + factor*dct_value, labels[i]))
        per_phoneme_values_supervectors.append((supervectors_value, labels[i]))
    decision_vectors_supervectors = compute_decision_vectors(per_phoneme_values_supervectors)
    decision_vectors_supervectors_dict[original_phoneme] = decision_vectors_supervectors
    decision_vectors_combination = compute_decision_vectors(per_phoneme_values_combination)
    mcnemar_dict[original_phoneme] = mcnemar(decision_vectors_supervectors, decision_vectors_combination)[1]

In [149]:
mcnemar_dict

{'B': 0.17956542968750003,
 'D': 0.13380050659179688,
 'G': 1.0,
 'N': 0.60723876953125011,
 'a': 0.18131184532598765,
 'b': 0.17968750000000003,
 'c': 0.078353881835937486,
 'd': 0.26493089646100998,
 'e': 0.17698343952928525,
 'g': 0.078353881835937486,
 'i': 0.0045116404083552176,
 'k': 0.13367423536410539,
 'l': 0.43970542918582195,
 'm': 0.40503224614076327,
 'n': 0.15457114715373352,
 'o': 0.65010561688412305,
 'p': 0.29121523654589498,
 'r': 0.3193082974752447,
 'rr': 0.74364418610520333,
 's': 0.093766736079311977,
 't': 0.029493493531317305,
 'u': 0.15240777196147803,
 'w': 1.0,
 'x': 0.125,
 'y': 0.37417441760470826,
 'z': 0.38769531250000011}

# Features combination

In [133]:
features_combination_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/final_experiments/heldout/mcnemar/mcnemar_vectors/features_combination/"
supervectors_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/final_experiments/heldout/mcnemar/mcnemar_vectors/supervectors/"

In [134]:
phonemes_features_combination = ["c", "x", "G", "d", "b", "g", "z", "w", "N", "B", "rr", "u", "p", "D", "y", "k", "m", "t", "l", "i", "r", "n", "s", "o", "a", "e"]

In [135]:
mcnemar_dict_features_combination = all_phonemes_empty_dict(phonemes_features_combination)
mcnemar_dict_2_features_combination = all_phonemes_empty_dict(phonemes_features_combination)
decision_vectors_features_combination_dict = all_phonemes_empty_dict(phonemes_features_combination)
decision_vectors_supervectors_2_dict = all_phonemes_empty_dict(phonemes_features_combination)

In [136]:
sets = ['e', 'r', 'a', 'o', 'c_x_d_G_b_g', 'k_m_t', 'l_i', 'p_D_y', 'r', 's', 'z_w_N_B_rr_u', 'n']

In [137]:
for s in sets:
    filename = features_combination_dir + s
    with open(filename, "rb") as f:
        s_dict = cPickle.load(f)
        for k in s_dict.keys():
            decision_vectors_features_combination_dict[k] = s_dict[k]

In [138]:
for s in sets:
    filename = supervectors_dir + s
    with open(filename, "rb") as f:
        s_dict = cPickle.load(f)
        for k in s_dict.keys():
            decision_vectors_supervectors_2_dict[k] = s_dict[k]

In [139]:
for phoneme in phonemes_features_combination:
    decision_vectors_supervectors = decision_vectors_supervectors_dict[phoneme]
    decision_vectors_supervectors_2 = decision_vectors_supervectors_2_dict[phoneme] 
    decision_vectors_features_combination = decision_vectors_features_combination_dict[phoneme]
    mcnemar_dict_features_combination[phoneme] = mcnemar(decision_vectors_supervectors, decision_vectors_features_combination)[1]
    mcnemar_dict_2_features_combination[phoneme] = mcnemar(decision_vectors_supervectors_2, decision_vectors_features_combination)[1]

In [140]:
mcnemar_dict_features_combination

{'B': 0.77441406250000033,
 'D': 1.0,
 'G': 1.0,
 'N': 0.99999999999999933,
 'a': 0.093477470231310283,
 'b': 1.0,
 'c': 0.87462931238042085,
 'd': 0.043285250663757359,
 'e': 0.2260801690470079,
 'g': 1.0,
 'i': 0.61844546603881445,
 'k': 0.74364418610520333,
 'l': 0.0011686374974383242,
 'm': 0.16188140552559774,
 'n': 0.85286561860668608,
 'o': 0.2765531697433502,
 'p': 1.0,
 'r': 0.0080875894711945062,
 'rr': 0.82214424245218543,
 's': 0.90914639012300891,
 't': 0.0042724050316516708,
 'u': 0.65296437551645969,
 'w': 0.548828125,
 'x': 1.0,
 'y': 0.16825920465992439,
 'z': 0.00085544586181640603}

# Supervectors sanity checks

In [141]:
decision_vectors_supervectors_dict_2 = {}
mcnemar_vectors_supervectors_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/final_experiments/heldout/mcnemar/mcnemar_vectors/supervectors/"

In [143]:
#for phoneme in phonemes_features_combination:
#    mcnemar_filename = mcnemar_vectors_supervectors_dir + phoneme
#    with open(mcnemar_filename, "rb") as f:
#        d = cPickle.load(f)
#        decision_vectors_supervectors_dict_2[phoneme] = d[phoneme]

In [144]:
for phoneme in phonemes_features_combination:
    print all((decision_vectors_supervectors_dict[phoneme] == decision_vectors_supervectors_2_dict[phoneme]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


# Phonemes differences

In [425]:
s1 = set(mcnemar_dict_features_combination.keys())
s2 = set(phonemes)

In [426]:
s2.difference(s1)

{'Y', 'c', 'f'}